# Updated version of distance measurement and analysis between macrophages and ECs

- New 'shortest distance' code
- New 'macs within radius' code
- Added randomised control

In [1]:
### LIBRARY
import os
import glob
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from skimage.io import imsave
from skimage.color import rgb2gray
from skimage.util import invert
from skimage.filters import try_all_threshold
from skimage.filters import threshold_minimum, threshold_isodata, threshold_triangle, threshold_mean
from skimage.filters import threshold_otsu, threshold_li, threshold_yen
from skimage.morphology import binary_dilation
from scipy.ndimage import binary_erosion
from scipy.ndimage import distance_transform_edt
from skimage.segmentation import watershed
from skimage.filters import gaussian
from skimage.morphology import label
from skimage.measure import regionprops
from skimage.segmentation import clear_border
from skimage.segmentation import relabel_sequential
from skimage.morphology import remove_small_objects
from skimage.morphology import remove_small_holes
from skimage.morphology import dilation, disk
from scipy.ndimage import binary_dilation
from scipy.spatial.distance import cdist

from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage import measure
from skimage.segmentation import watershed

from statistics import mean, stdev
import re
from natsort import natsorted

from skimage import exposure
from skimage.morphology import closing

In [26]:
### PARAMETERS
filename = 'C:/Users/terez/Documents/Skola/Ludwig-Maximilians_Universitat_Munchen/Research course Moretti lab/Microscopy data etc/EXP6/Macrophages_and_ECs/ImmEpis_EXP6_d3_GFP_cTnT_CD31'
pixel_size = 0.65

In [27]:
## separated for the papermil

# filename extract
last_folder = os.path.basename(filename)
# output folder:
output_folder = 'C:/Users/terez/Documents/Skola/Ludwig-Maximilians_Universitat_Munchen/Research course Moretti lab/Microscopy data etc/EXP6/EC_analysis/Distances'

# to save masks:
# to save masks:

folder = f'{filename}/masks_distances'
folder1 = f'{filename}/masks_with_nuclei_count'
folder2 = f'{filename}/masks_nuclei'
os.makedirs(folder1, exist_ok=True)
os.makedirs(folder2, exist_ok=True)
os.makedirs(folder, exist_ok=True)

print(last_folder)

ImmEpis_EXP6_d3_GFP_cTnT_CD31


In [28]:
### LOADING

## DAPI in 405 channel:
list_of_dapi_files = natsorted(glob.glob(f"{filename}/*ch00*.tif"))
images_dapi = {}
for file in list_of_dapi_files:
    img = io.imread(file)
    images_dapi[os.path.basename(file)] = img

## 488 channel:
list_of_488_files = natsorted(glob.glob(f"{filename}/*ch01*.tif"))
images_488 = {}
for file in list_of_488_files:
    img = io.imread(file)
    images_488[os.path.basename(file)] = img

## 594 channel:
list_of_594_files = natsorted(glob.glob(f"{filename}/*ch02*.tif"))
images_594 = {}
for file in list_of_594_files:
    img = io.imread(file)
    images_594[os.path.basename(file)] = img

## 647 channel:
list_of_647_files = natsorted(glob.glob(f"{filename}/*ch03*.tif"))
images_647 = {}
for file in list_of_647_files:
    img = io.imread(file)
    images_647[os.path.basename(file)] = img

print(f"to check: \n number of 405 images: \t {len(images_dapi)}, \n number of 488 images: \t {len(images_488)},"
      f"\n number of 594 images: \t {len(images_594)}, \n number of 647 images: \t {len(images_647)}")

to check: 
 number of 405 images: 	 15, 
 number of 488 images: 	 15,
 number of 594 images: 	 15, 
 number of 647 images: 	 15


In [29]:
## Grabbing the cellpose masks from the folder

list_of_nuclei_mask_files = natsorted(glob.glob(f"{folder2}/*.tif"))
nuclei_masks = {}

for file in list_of_nuclei_mask_files:
    img = io.imread(file)
    nuclei_masks[os.path.basename(file)] = img

print(f'Number of nuclei masks: {len(nuclei_masks)}')

Number of nuclei masks: 15


In [30]:
### Channel gating

tested_image = 'Image 1' #d1_6

## testing:
# tries all automatic thresholding options for the image 
# fig, ax = try_all_threshold(blurred_dapi_test, figsize=(6, 12), verbose=False)
# plt.show()

## DAPI/organoid area:
def dapi_analysis(mask):
    thresholding = mask > threshold_triangle(mask)
    dilated = binary_dilation(thresholding, iterations=100) #dilates the individual nuclei by 100 pixels
                                                            #this covers up the holes/spaces in between
                                                            #also bridges whatever cavities etc. are present 
    thresholding_2 = remove_small_holes(dilated, area_threshold=50000) #removes the remaining holes
    eroded = binary_erosion(thresholding_2, iterations=80) #erodes the expanded mask back to original size
                                                           #basically 'tightens' the mask to better fit the 
                                                           #organoid
    thresholding_3 = remove_small_objects(eroded, min_size=8000) #removes stray nuclei outside of the organoid
    blurred = gaussian(thresholding_3, sigma=10) #blurrs the jagged edges
    smoothed = blurred > 0.2 #blurring makes the image non-binary, this makes it binary again by selecting
                             #the intensities over certain threshold (in this case everything over 20%)

    return(smoothed)


## 488
def analysis_488(image,object_min_size):
    thresholding_1 = image > threshold_triangle(image) #originaly otsu
    thresholding_2 = remove_small_objects(thresholding_1, min_size=object_min_size) #100
    return(thresholding_2)

def watershed_488(mask):
    #new:
    distance_transform_488 = ndi.distance_transform_edt(mask)
    # Find peaks (likely centers of individual cells)
    local_maxi_test = peak_local_max(distance_transform_488, labels=mask, footprint=np.ones((3, 3)), min_distance=30) #originaly 30
    local_maxi_test_2 = np.zeros_like(distance_transform_488, dtype=bool)
    local_maxi_test_2[tuple(local_maxi_test.T)] = True

    # Label the peaks
    markers_test = measure.label(local_maxi_test_2)

    # watershed
    watershed_labels_test = watershed(-distance_transform_488, markers_test, mask=mask)

    return(watershed_labels_test)


## 647
def analysis_647(image):
    background = gaussian(image, sigma=50)   # smooth background
    normalisation = image - background
    normalisation = exposure.rescale_intensity(normalisation, out_range=(0, 1))
    thresholding_1 = normalisation > threshold_triangle(normalisation) # originaly triangle / 470
    thresholding_2 = remove_small_objects(thresholding_1, min_size=100)

    # thresholding_2 = remove_small_objects(thresholding_1,min_size=object_min_size)
    # dilated = binary_dilation(thresholding_1, iterations=15)
    # thresholding_3 = remove_small_objects(dilated, min_size=100) #originaly 250
    # erosion = binary_erosion(thresholding_3, iterations=14)
    # blurred = gaussian(erosion, sigma=3) #blurrs the jagged edges
    # smoothed = blurred > 0.5
    return(thresholding_2)

In [31]:
Image_data_CD31_files = {}
Individual_data_CD31_files = {}

min_size_488 = 100
mac_radius = 13

for name,img in images_488.items():

    ### CREATION OF INDIVIDUAL IMAGE MASKS:

    ## 488
    # img_488 = images_488[f'{name_without_last_part}_ch01.tif']
    thresholded_488 = analysis_488(img, object_min_size=min_size_488)
    watersheded_488 = watershed_488(thresholded_488)
    objects_488 = regionprops(watersheded_488,img)

    num_macrophages = watersheded_488.max()

    #image name extracted:
    name_without_last_part = '_'.join(name.split('_')[:-1]) 
    print(name_without_last_part)

    ## DAPI
    dapi_mask = nuclei_masks[f'{name_without_last_part}_ch00_mask.tif']
    thresholded_405 = dapi_analysis(dapi_mask)
    organoid_size = np.sum(thresholded_405)
    organoid_size_in_um = organoid_size*(pixel_size**2)


    ## 647
    img_647 = images_647[f'{name_without_last_part}_ch03.tif']
    thresholded_647 = analysis_647(img_647) & thresholded_405
    labelled_647 = label(thresholded_647)
    objects_647 = regionprops(labelled_647,img_647)

    num_CD31_cells = labelled_647.max()

    ## fake 488 mask
    # this determines where the pixels are positive in the dapi mask:
    organoid_coords_2 = np.column_stack(np.where(thresholded_405))
    total_points_2 = organoid_coords_2.shape[0]
    # this selects random coordinates inside the positive area of the dapi mask that is equal to the number 
    # of macs detected (approximated since the macs are basically a single mass and not individual cells)
    select_indeces_2 = np.random.choice(total_points_2, size=num_macrophages, replace=False)
    selected_coords_2 = organoid_coords_2[select_indeces_2]
    # create an empty mask
    expanded_mask = np.zeros_like(thresholded_405, dtype=bool)
    # structuring element
    fake_mac = disk(mac_radius)
    # new 'fake' mac mask:
    seed_mask = np.zeros_like(thresholded_405, dtype=bool)
    seed_mask[selected_coords_2[:,0], selected_coords_2[:,1]] = 1
    dilated = binary_dilation(seed_mask, fake_mac)
    fake_mac_mask = np.logical_and(dilated, thresholded_405)


    ### HERE THE ANALYSIS STARTS

    number_of_iter = 46 #29.9um

    #this is for the real data:
    original_areas = []
    nearest_distances = []
    mac_area_around_EC = []

    # this is for the randomised control:
    mac_radius = 13
    random_original_areas = []
    random_nearest_distances = []
    random_mac_area_around_EC = []

    for obj in objects_647:
        
        obj_mask = np.zeros_like(thresholded_488, dtype=bool)
        obj_mask[tuple(obj.coords.T)] = True

        original_area = obj.area*(pixel_size**2)
        original_areas.append(original_area)

        ## this is for the real data:
        #nearest distance:
        dilated_obj = obj_mask.copy()

        for i in range(1, number_of_iter + 1):
            dilated_obj = binary_dilation(dilated_obj, iterations=1)
            overlap = np.any(dilated_obj & (thresholded_488>0))
            if overlap:
                nearest_distances.append(i*pixel_size)
                break
        else:
            nearest_distances.append(number_of_iter+1)

        #mac overlap:
        dilated_obj_2 = binary_dilation(obj_mask.copy(), iterations=number_of_iter)
        mac_area_in_radius = np.sum(thresholded_488 & dilated_obj_2)
        mac_area_around_EC.append(mac_area_in_radius*(pixel_size**2))

        ##this is for randomised control:
        dilated_obj_3 = obj_mask.copy()

        for i in range(1, number_of_iter + 1):
            dilated_obj_2 = binary_dilation(dilated_obj_3, iterations=1)
            overlap = np.any(dilated_obj_3 & (fake_mac_mask>0))
            if overlap:
                random_nearest_distances.append(i*pixel_size)
                break
        else:
            random_nearest_distances.append(number_of_iter+1)

        #mac overlap:
        dilated_obj_4 = binary_dilation(obj_mask.copy(), iterations=number_of_iter)
        random_mac_area_in_radius = np.sum(fake_mac_mask & dilated_obj_4)
        random_mac_area_around_EC.append(random_mac_area_in_radius*(pixel_size**2))

    per_of_unmac_ECs = num_CD31_cells/nearest_distances.count(47)
    per_of_unmac_ECs_ctrl = num_CD31_cells/random_nearest_distances.count(47)

    average_min_dist = mean(nearest_distances)
    average_min_dist_ctrl = mean(random_nearest_distances)

    average_mac_area = mean(mac_area_around_EC)
    average_mac_area_ctrl = mean(random_mac_area_around_EC)

    average_CD31_area = mean(original_areas)

    Image_data_CD31_files[name_without_last_part] = {
        'CD31_number': num_CD31_cells,
        'CD31 average area': average_CD31_area,
        'Mac number': num_macrophages,
        'EC w/o mac [%]': per_of_unmac_ECs,
        'EC w/o mac ctrl [%]': per_of_unmac_ECs_ctrl,
        'Average min distance [um]': average_min_dist,
        'Average min distance ctrl [um]': average_min_dist_ctrl,
        'Average mac area in radius [um^2]': average_mac_area,
        'Average mac area in radius ctrl [um^2]': average_mac_area_ctrl
    }

    Individual_data_CD31_files[name_without_last_part] = {
        'CD31_areas': original_areas,
        'Minimal distances': nearest_distances,
        'Minimal distances ctrl': random_nearest_distances,
        'Mac area in radius': mac_area_around_EC,
        'Mac area in radius ctrl': random_mac_area_around_EC
    }


Image 1
Image 2
Image 3
Image 4
Image 5
Image 6
Image 7
Image 8
Image 9
Image 10
Image 11
Image 12
Image 13
Image 15


C:\Users\terez\AppData\Local\Temp\ipykernel_3088\4289361689.py:115: RuntimeWarning: divide by zero encountered in scalar divide
  per_of_unmac_ECs = num_CD31_cells/nearest_distances.count(47)


Image 16


In [32]:
### DATAFRAME
df_cd31_distances = pd.DataFrame.from_dict(Image_data_CD31_files, orient='index')
output_path = f'{output_folder}/{last_folder}_distances.xlsx'
df_cd31_distances.to_excel(output_path)

# New data frame:
# Individual_data_CD31_files_2 = {}

# for name, variables in Individual_data_CD31_files.items():
#     for var_name, var_list in variables.items():
#         flat_key = f'{name}_{var_name}'
#         Individual_data_CD31_files_2[flat_key] = var_list
# df_individual_distances = pd.DataFrame.from_dict(Individual_data_CD31_files_2,orient='index') #
# reversed_df_individual_distances = df_individual_distances.T

output_path_2 = f'{output_folder}/{last_folder}_individual_data.xlsx'
# reversed_df_individual_distances.to_excel(output_path_2)

# reversed_df_individual_distances.head()

# v2
with pd.ExcelWriter(output_path_2) as writer:
    for name,variables in Individual_data_CD31_files.items():
        df = pd.DataFrame(variables)
        df.to_excel(writer,sheet_name=name, index=False)